In [ ]:
from google.colab import drive  
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Importando librerias
import random
import json
import pickle
import numpy as np 

#Importar librerias necesarias para lematizar
import nltk

from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('omw-1.4')
#Importar librerias necesarias para la red neuronal
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
#Definiendo la funcion de  lematizacion de raiz
lemmatizer = WordNetLemmatizer()

In [ ]:
#Lectura del archivo json para la bolsa de palabras
intents = json.loads(open('/content/drive/MyDrive/topicos/QUECHUA.json').read())

In [ ]:
words = []
classes = []
documents = []
ignore_letters = ['?','!','.',',']

for intent in intents ['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if  intent['tag'] not in classes:
            classes.append(intent['tag'])


In [ ]:
#Eliminar duplicados y lematizar
nltk.download('wordnet')
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]

words = sorted(set(words))
classes = sorted(set(classes))

pickle.dump(words, open('words.pk1','wb'))
pickle.dump(classes, open('classes.pk1','wb'))


[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
#Creamos una bolsa de palabras
training = []
output_empty = [0]*len(classes)
for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

In [ ]:
#Entrenando el modelo
random.shuffle(training)
training = np. array(training)

#Definiendo valores de enternamiento
train_x = list(training[:,0])
train_y = list(training[:,1])


<ipython-input-8-040c47b20e04>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np. array(training)


In [ ]:
#Creando una red neural

model = Sequential()
model.add(Dense(64,input_shape=(len(train_x[0]),),activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation = 'softmax'))

sgd = SGD(lr=0.01, decay= 1e-6,momentum = 0.9, nesterov = True)

#Defininendo la compilacion y la parte que se entrenará
model.compile(loss = 'categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.fit(np.array(train_x),np.array(train_y),epochs=256, batch_size =5)
model.save('chatbot_model.model')
print('hecho')

Epoch 1/256


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


27/27 [==============================] - 2s 3ms/step - loss: 2.7935 - accuracy: 0.0746
Epoch 2/256
27/27 [==============================] - 0s 3ms/step - loss: 2.7420 - accuracy: 0.0896
Epoch 3/256
27/27 [==============================] - 0s 3ms/step - loss: 2.7152 - accuracy: 0.1642
Epoch 4/256
27/27 [==============================] - 0s 3ms/step - loss: 2.6802 - accuracy: 0.1418
Epoch 5/256
27/27 [==============================] - 0s 3ms/step - loss: 2.7070 - accuracy: 0.1343
Epoch 6/256
27/27 [==============================] - 0s 3ms/step - loss: 2.6586 - accuracy: 0.1716
Epoch 7/256
27/27 [==============================] - 0s 3ms/step - loss: 2.5918 - accuracy: 0.1791
Epoch 8/256
27/27 [==============================] - 0s 3ms/step - loss: 2.6199 - accuracy: 0.1418
Epoch 9/256
27/27 [==============================] - 0s 3ms/step - loss: 2.5139 - accuracy: 0.2090
Epoch 10/256
27/27 [==============================] - 0s 3ms/step - loss: 2.5541 - accuracy: 0.1940
Epoch 11/256
27/27 [=

In [ ]:
model.save('/content/drive/MyDrive/topicos/chatbot_model.h5')

In [ ]:
#Importar librerias para el modelo de chatbot
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model

In [ ]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('/content/drive/MyDrive/topicos/QUECHUA.json').read())

#Cargando archivos de forma binaria
words = pickle.load(open('words.pk1','rb'))
classes = pickle.load(open('classes.pk1','rb'))

#Cargamos el modelo
model = load_model('chatbot_model.model')
 

In [ ]:
#Funcion para limpiar oraciones
def clean_up_sentences(sentece):
    sentece_words = nltk.word_tokenize(sentece)
    sentece_words = [lemmatizer.lemmatize(word) for word in sentece_words]
    return sentece_words

def bag_of_words(sentece):
    sentece_words = clean_up_sentences(sentece)
    bag = [0]* len(words)
    for w in sentece_words:
        for i, word in enumerate (words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

In [ ]:
#Funcion para predecir respuesta
def predict_class(sentece):
    bow = bag_of_words(sentece)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    result = [[i,r] for i, r in enumerate(res) if r > ERROR_THRESHOLD ]

    result.sort(key =lambda x: x[1],reverse= True)
    return_list=[]
    for r in result:
        return_list.append({'intent':classes[r[0]],'probability':str(r[1])})
    return return_list

**Uso del chatbot**

In [ ]:
#Funcion de respuesta e interfaz del bot
import pandas as pd
def get_response(intent_list,intent_json):
    tag = intent_list[0]['intent']
    list_of_intents = intent_json['intents']
    for i in  list_of_intents:
        if i['tag'] == tag:
            result =random.choice(i['responses'])
            break
    return result

print("Comienza a conversar con el bot")
acum = []
acum2 = []
contar = 0
while True:

    message = input("")
    if message == 'salir':
      break
    ints = predict_class(message)
    res = get_response(ints,intents)
    print(res)
    contar = contar + 1
 
    acum = acum +[message]
    
#Se crea un registro de todo lo hablado
data = {'mensaje': acum}
# df1 = pd.DataFrame(data, columns = ['mensaje'])
#df1.to_csv('example.csv')



Comienza a conversar con el bot
hola
1/1 [==============================] - 0s 16ms/step
Hola :)
hola
1/1 [==============================] - 0s 13ms/step
Hola! kusikuy riqsirikuyk(Hola! un gusto conocerte)
como estas
1/1 [==============================] - 0s 20ms/step
kusikuywan kamachikusqaykita rurasaq, ichaqa qam imaynallataq tarikunki? (Feliz de estar a tu disposición,  ¿como te encuentras tú?)
estoy triste
1/1 [==============================] - 0s 31ms/step
 imapis kanman ñuqa qanwan kachkasaq(Si necesitas apoyo puedes contar conmigo)
salir


In [ ]:
sentence = ', '.join(data['mensaje'])
sentence

'hola, hola, como estas, estoy triste'

In [ ]:
from textblob import TextBlob

In [ ]:
def get_sentiment (polarity):
  if polarity < -0.2:
    return 'negativo'
  elif polarity > 0.2:
    return 'positivo'
  else:
    return 'neutro'

In [ ]:
res = TextBlob(sentence).translate(from_lang='es', to='en').sentiment
tmp = get_sentiment(res.polarity)
print(sentence, ': ', res.polarity, ' => ', tmp)

hola, hola, como estas, estoy triste :  -0.5  =>  negativo


In [ ]:
for i in range(len(data['mensaje'])):
  aux = TextBlob(data['mensaje'][i]).translate(from_lang='es', to='en').sentiment
  tmp = get_sentiment(aux.polarity)
  print(data['mensaje'][i], ': ', aux.polarity, ' => ', tmp)

hola :  0.0  =>  neutro
hola :  0.0  =>  neutro
como estas :  0.0  =>  neutro
estoy triste :  -0.5  =>  negativo
